In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-08-11 20:38:21 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-08-11 20:38:22 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-08-11 20:38:22 __main__ INFO     transformers.__version__='4.54.1'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-08-11 20:38:24 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-08-11 20:38:24 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-08-11 20:38:25 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-08-11 20:38:25 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-08-11 20:38:25 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-08-11 20:38:25 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-08-11 20:38:25 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-08-11 20:38:25 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-08-11 20:38:25 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.3-70B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-08-11 20:39:15 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-08-11 20:39:15 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-08-11 20:39:15 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


## Selection Task

In [5]:
from src.selection.data import SelectOneTask

select_prof = SelectOneTask.load(
    path="/disk/u/arnab/Codes/Projects/retrieval/data_save/selection/profession.json"
)

print(select_prof)

SelectOneTask: (profession of a famous person)
Categories: actor(20), singer(20), comedian(20), director(20), basketball player(20), football player(20), soccer player(20), tennis player(20), golfer(20), boxer(20), news anchor(20), journalist(20), author(20), fashion designer(20), entrepreneur(19), politician(20)



In [25]:
from typing import Literal

output_formatting: Literal[
    "zero_shot", # no formatting, model preference
    "object", # Bill Gates
    "lettered", # a. Bill Gates
] = "lettered"

In [50]:
prompt_template_idx = 1
option_style = "numbered"  # "numbered", "lettered", "ordinal"

# one_shot = select_prof.get_random_sample(
#     mt = mt,
#     prompt_template_idx=prompt_template_idx,
#     option_style="numbered",
#     category="actor",
#     filter_by_lm_prediction = False,
# )

# print(one_shot)

sample = select_prof.get_random_sample(
    mt = mt,
    obj_idx=3,
    prompt_template_idx=prompt_template_idx,
    option_style=option_style,
    category="actor",
    filter_by_lm_prediction = True,
    # output_formatting=output_formatting,
)
print(sample)

Denzel Washington -> Julia Roberts (3): ['Colleen Hoover', 'Scottie Scheffler', "Bill O'Reilly", 'Julia Roberts', 'Jude Bellingham', 'Anthony Joshua']


In [51]:
# sample.prompt_template = select_prof.prompt_templates[1]

print(f'"{sample.prompt()}"', ">>", sample.obj)

"a. Colleen Hoover
b. Scottie Scheffler
c. Bill O'Reilly
d. Julia Roberts
e. Jude Bellingham
f. Anthony Joshua
Who among these people mentioned above share the same profession as Denzel Washington?
Answer:" >> Julia Roberts


In [52]:
# print(sample.prompt(option_style="single_line"))

In [53]:
from src.functional import generate_with_patch

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)[0]
print(f'"{gen}"', ">>", sample.obj)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


" d. Julia Roberts
Explanation: Denzel Washington is an actor, and Julia Roberts is also an" >> Julia Roberts


In [55]:
from src.functional import prepare_input, get_hs, interpret_logits
from src.selection.utils import get_first_token_id, verify_correct_option

# inputs = prepare_input(prompts=sample.prompt(), tokenizer=mt)
# logit_module = (mt.lm_head_name, -1)
# logits = get_hs(
#     mt=mt,
#     input=inputs,
#     locations=[logit_module],
#     return_dict=False,
# ).squeeze()

verify_correct_option(
    mt=mt,
    # logits=logits,
    target=sample.obj,
    options=sample.options,
    input=sample.prompt()
)

(True,
 [PredictedToken(token=' d', prob=0.4921875, logit=20.5, token_id=294, metadata=None),
  PredictedToken(token=' (', prob=0.1591796875, logit=19.375, token_id=320, metadata=None),
  PredictedToken(token=' Julia', prob=0.140625, logit=19.25, token_id=40394, metadata=None),
  PredictedToken(token=' Only', prob=0.05859375, logit=18.375, token_id=8442, metadata=None),
  PredictedToken(token=' The', prob=0.0517578125, logit=18.25, token_id=578, metadata=None)],
 {40394: (3,
   PredictedToken(token=' Julia', prob=0.140625, logit=19.25, token_id=40394, metadata=None)),
  4349: (8,
   PredictedToken(token=' Col', prob=0.00701904296875, logit=16.25, token_id=4349, metadata=None)),
  21353: (126,
   PredictedToken(token=' Anthony', prob=2.372264862060547e-05, logit=10.5625, token_id=21353, metadata=None)),
  8766: (146,
   PredictedToken(token=' Bill', prob=1.9669532775878906e-05, logit=10.375, token_id=8766, metadata=None)),
  62734: (178,
   PredictedToken(token=' Jude', prob=1.442432403

In [29]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.subj,
    mt = mt,
    heads = HEADS
)

0 patches to ablate possible answer information from options
2025-08-11 14:11:43 src.experiments.utils DEBUG    Predictions: ['" d"[294] (p=0.980, logit=26.875)', '" a"[264] (p=0.008, logit=22.000)', '" f"[282] (p=0.004, logit=21.250)', '" b"[293] (p=0.003, logit=21.125)', '" c"[272] (p=0.002, logit=20.750)']
2025-08-11 14:11:43 src.experiments.utils INFO     Combined attention matrix for all heads


## Counting Task

In [ ]:
from src.selection.data import CountingTask

counting_fruits = CountingTask.load(
    path="../data_save/counting/fruits.json"
)

print(counting_fruits)

In [ ]:
sample = counting_fruits.get_random_sample(
    mt = mt,
    prompt_template_idx=0,
    option_style="single_line",
    category="fruits",
    filter_by_lm_prediction=True,
    n_count=2,
    n_distractors=3
)

print(sample)

In [ ]:
sample.prompt_template = counting_fruits.prompt_templates[1]

print(f'"{sample.prompt()}"', ">>", sample.count)

In [ ]:
from src.functional import generate_with_patch

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)[0]
print(f'"{gen}"', ">>", sample.count)

In [ ]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.category,
    mt = mt,
    heads = HEADS
)

## Deduction Task

In [ ]:
from src.selection.data import DeductionTask

deduction_task = DeductionTask.load(
    dir_path="../data_save/deduction"
)

print(deduction_task)

In [ ]:
sample = deduction_task.get_random_sample(
    mt = mt,
    topic_name = "height",
    depth = 5,
)

print(sample)

In [ ]:
from src.functional import generate_with_patch

generate_with_patch(
    mt = mt,
    inputs = sample.prompt,
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)

In [ ]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt,
    options = ["Alice", "Bob", "Cam", "Dave", "Eli"],
    pivot = sample.answer,
    mt = mt,
    heads = HEADS
)

## All of the Above Selection

In [ ]:
from src.selection.data import SelectAllTask

select_all_prof = SelectAllTask.load(
    path="../data_save/selection/profession.json"
)

print(select_all_prof)

In [ ]:
sample = select_all_prof.get_random_sample(
    mt=mt
)

print(sample)

In [ ]:
from src.functional import generate_with_patch

generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    n_gen_per_prompt=1,
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)

In [ ]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.category,
    mt = mt,
    heads = HEADS
)